# Assignment-04 基于维基百科的词向量构建¶

## Step -01 Download the Wikipedia Chinese Corpus 



## Step-02: Using https://github.com/attardi/wikiextractor to extract the wikipedia corpus
第二步：使用python wikipedia extractor抽取维基百科的内容

用 wikiextractor.py 对内容进行抽取，得到wiki.txt

然后将文本里的繁体都转换为简体，使用opencc，需要安装opencc (https://github.com/yichen0831/opencc-python ) 

提取文本里的中文，去除英文内容，并且进行分词处理，得到wiki_only_simple_and_cut.txt

In [1]:

import logging
import sys
#from gensim.corpora import WikiCorpus


In [1]:
pwd

'C:\\Users\\herbi\\Google Drive\\Current_NLP_Bootcamp\\nlp_hw_notes'

In [2]:
path_to_wiki_dump = "C:\\Users\\herbi\\Documents\\nlp_camp_local\\zhwiki-20190720-pages-articles-multistream.xml.bz2"
path_to_wiki_folder="C:\\Users\\herbi\\Documents\\nlp_camp_local"
wiki_dump="zhwiki-20190720-pages-articles-multistream.xml.bz2"

In [3]:
import os 
os.chdir(path_to_wiki_folder)

In [7]:
pwd

'C:\\Users\\herbi\\Documents\\nlp_camp_local'

Use WikiExtractor.py to extract all the wiki articles 
https://github.com/attardi/wikiextractor/wiki#Usage 



In [ ]:
!python   WikiExtractor.py   zhwiki-20190720-pages-articles-multistream.xml.bz2
 

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import jieba
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from tqdm import tqdm
from pathlib import Path
from opencc import OpenCC
import os
import re
import pickle

In [5]:
%matplotlib notebook

In [6]:
 
zhwiki = Path('C:\\Users\\herbi\\Documents\\nlp_camp_local\\text_v0\\AA')

In [11]:
def trad2sim(input_dir, output_dir):
    output_dir = input_dir.parent / output_dir
    if not output_dir.exists(): output_dir.mkdir(parents=True, exist_ok=True)
    for wiki in tqdm(input_dir.iterdir()):
        os.system('python -m opencc -c t2s -i {wiki} -o {output}'.format(
            wiki=str(wiki), output=str(output_dir/('simplified_'+wiki.name))
        ))

In [16]:
trad2sim(zhwiki,'simplified_zhwiki')

100it [02:21,  1.37s/it]


#### note: tqdm : https://github.com/tqdm/tqdm 
Progress bar

## load stop words 

In [12]:
stop_words_set='的'

## 分词

In [13]:
simplified_zhwiki = zhwiki.parent / 'simplified_zhwiki'
simplified_zhwiki.exists()

True

In [14]:
def generate_texts(simplified_zhwiki):
    texts = []
    for wiki in tqdm(simplified_zhwiki.iterdir()):
        with wiki.open(encoding='utf8') as f:
            for line in f.readlines():
                line = line.strip()
                if line == '' or line.startswith('<doc') or line.startswith('</doc'): continue
                words = []
                for word in jieba.cut(line):
                    if word in stop_words_set: continue
                    words.append(word)
                if len(words) > 1: texts.append(words.copy())
    return texts

In [17]:
texts = generate_texts(simplified_zhwiki)

0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\herbi\AppData\Local\Temp\jieba.cache
Loading model cost 0.691 seconds.
Prefix dict has been built succesfully.
100it [04:41,  3.27s/it]


In [18]:
len(texts)

283210

In [19]:
with open('corpus.pickle', 'wb') as f:
    pickle.dump(texts, f)

In [21]:
pwd

'C:\\Users\\herbi\\Documents\\nlp_camp_local'

## Word2vec
Step-03: Using gensim get word vectors:
Reference:

https://radimrehurek.com/gensim/models/word2vec.html
https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne
第三步：参考Gensim的文档和Kaggle的参考文档，获得词向量。 注意，你要使用Jieba分词把维基百科的内容切分成一个一个单词，然后存进新的文件中。然后，你需要用Gensim的LineSentence这个类进行文件的读取。

训练成词向量Model.

In [7]:
import pickle 
from gensim.models import Word2Vec

In [8]:
with open('corpus.pickle', 'rb') as f:
    corpus = pickle.load(f)

In [10]:
len(corpus)

283210

In [11]:
corpus_short=corpus[:2000]

In [12]:
model = Word2Vec(sentences=corpus_short, size=300, window=5, min_count=3, workers=4)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [13]:
model.wv['数学']

array([-7.45744705e-02, -3.01090389e-01,  9.09206793e-02,  3.02521378e-01,
        2.42115304e-01, -2.21010357e-01, -7.73816481e-02, -3.28866094e-01,
       -1.18302405e-01,  4.72516604e-02, -1.39063746e-01, -5.61035164e-02,
       -1.17170028e-01,  2.30280355e-01,  2.14368880e-01,  2.28909150e-01,
        3.28356713e-01, -4.09788787e-02,  5.15622133e-03,  2.18565539e-02,
        1.65645555e-01, -1.54658034e-01,  2.26505503e-01,  1.95999257e-02,
       -8.36538076e-02, -1.32283103e-02, -5.37586436e-02,  1.32813022e-01,
       -3.10738951e-01,  3.18564773e-01,  2.38405913e-01, -4.58056480e-01,
       -2.33745515e-01, -5.31336516e-02, -1.43277705e-01,  2.43143402e-02,
        1.43233031e-01,  2.67610699e-01, -1.73286796e-01, -2.43222862e-01,
       -5.74793033e-02, -1.39925167e-01, -2.95943078e-02,  4.18826938e-01,
        1.21859647e-01,  3.40619832e-01, -1.50585413e-01, -2.43832678e-01,
        1.30350798e-01, -2.12694302e-01,  6.81144446e-02, -2.55437553e-01,
        2.36266986e-01, -

In [14]:
model.save('shortcorpus_word2vec.model')

In [17]:
model.most_similar('数学')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('新', 0.9998358488082886),
 ('现实', 0.9996786117553711),
 ('问题', 0.9995670318603516),
 ('物理', 0.9995082020759583),
 ('文明', 0.9994291067123413),
 ('无', 0.9994125366210938),
 ('竞争', 0.9993775486946106),
 ('观点', 0.9993489980697632),
 ('即使', 0.9993197917938232),
 ('过程', 0.9992666244506836)]

In [26]:
model.wv.distance('数学', '过程')

0.0007332563400268555